# Implementation

## Pythonic Nuances
- `ABC` stands for *$A$bstract $B$ase $C$lass*. This is the way I learned how to implement inheritance-based polymorphism
- `__getitem__` and `__setitem__` give you the conveniences of `self[index] = value` style syntax, matching the pseudocode in $CLRS$
- `x // y` is equivalent to $\lfloor x \div y \rfloor$ (aka. *floor division*)
- Python is a zero-indexed language; as such, the first element of a list is the $0^{th}$, not the $1^{st}$. As such, to do the floor division required by `_left(i), _right(i), _parent(i)`, we must do some rejiggering.
    - we must add 1 to i before we divide, so we can do our 1-indexed calculations on natural numbers ( ${\mathbb{N}}$ )
    - we must subtract 1 to convert back to our 0-indexed convention for indexing items of Python lists

In [1]:
from IPython.display import display_markdown
from abc import ABC, abstractmethod
from gvanim import Animation, gif, render
import enum
import hashlib
import inspect
import os


class Heap(ABC):
    def __init__(self, ls):
        self.ls = ls
        self.heapsize = 0
        self.length = len(ls)
        self.animation = Animation()
        self.retree_animation()

    def __getitem__(self, i):
        if not i < self.length:
            return None
        return self.ls[i]

    def __setitem__(self, i, val):
        self.ls[i] = val

    def __repr__(self):
        return f'{self.classname()}: {self.ls}, heapsize: {self.heapsize}'

    @abstractmethod
    def classname(self):
        ...

    @abstractmethod
    def heap_prop(self, i, force=False) -> (bool, int | None):
        '''
        Describes the appropriate relationship
        between parents and children in the heap.

        :param int i: parent index, to which derived child-indices are compared
        :param bool force: Describes whether to only look in the heap (False),
            or whether to look in the entirety of self.ls
        :return: a tuple `(ok, idx)`. `ok` describes whether the heap_property
            is satisfied (True), or violated (False). `idx` describes the index
            of the value that ought to be swapped for the value at self[i]
        :rtype: (bool, int)
        '''
        ...

    def heapify(self, i):
        '''
        Bubbles down values that do not abide
        by the self.heap_prop, starting at index i

        :param int i: parent index of the heap or subheap that will be walked.
        :param bool force: Describes whether to only look in the heap (False),
            or whether to look in the entirety of self.ls
        '''

        ok, idx = self.heap_prop(i)
        if not ok:
            self._swap(idx, i)
            self.heapify(idx)
        return self

    def build_heap(self):
        '''
        Transforms an unsorted self.ls into a heap
        that abides by self.heap_prop
        '''

        if self.animation.steps():
            self.animation = Animation()
        self.heapsize = self.length
        for i in reversed(range(0, self.length//2)):
            self.heapify(i)
        print(f'Op: {inspect.currentframe().f_code.co_name}, {self}')
        return self

    def heapsort(self):
        '''
        Takes self.heapsize from self.length to 1, while reordering self.ls
        MaxHeap will give an ascending total order.
        MinHeap will give a descending total order.
        '''

        self.build_heap()
        for i in reversed(range(1, self.length)):
            self._swap(0, i)
            self.heapsize -= 1
            self.heapify(0)
        self.untree_animation()
        print(f'Op: {inspect.currentframe().f_code.co_name}, {self}')
        return self

    def bubbleup(self, i, force=False):
        '''
        Recursively increases the height of a value in the heap, until its
        relation to its parent no longer violates self.heap_prop

        :param int i: index, from which a parent-index is derived
        '''

        p = self._parent(i, force=force)
        ok, idx = self.heap_prop(p, force=force)
        if not ok:
            assert idx
            self._swap(p, idx)
            self.bubbleup(p, force=force)
        return self

    def insert(self, val):
        '''
        Adds a new value to the heap, in a spot that satisfies self.heap_prop

        :param val: comparable value to insert
        '''

        assert self.length == self.heapsize, f'self.length: {
            self.length}, self.heapsize: {self.heapsize}'
        self.ls.append(val)
        self.length += 1
        self.bubbleup(self.heapsize, force=True)
        self.heapsize += 1
        print(f'Op: {inspect.currentframe().f_code.co_name}, {self}')
        return self

    def delete(self, i):
        '''
        Removes a value from the heap and rebalances the heap accordingly

        :param int i: index to remove
        '''

        self._swap(i, self.length-1)
        self.ls = self.ls[:-1]
        self.length = len(self.ls)
        p = self._parent(i)
        ok, idx = self.heap_prop(p)
        if not ok:
            assert idx
            self.bubbleup(i)
        else:
            self.heapify(i)
        self.heapsize = self.length
        print(f'Op: {inspect.currentframe().f_code.co_name}, {self}')
        return self

    def retree_animation(self):
        '''
        Resets the animation frame to the current state of self.ls,
        represented as a tree
        '''

        self.animation.next_step(clean=True)
        for i, v in enumerate(self.ls):
            self.animation.label_node(i, label=v)
            p = self._parent(i)
        # if p and i != p:
            self.animation.add_edge(p, i)

    def untree_animation(self):
        '''
        Resets the animation frame to the current state of self.ls,
        represented as an array
        '''

        self.animation.next_step(clean=True)
        for p, v in enumerate(self.ls):
            i = p + 1
            self.animation.label_node(i, label=v)
            if self.animation.graphs()[-1].find(f'label="{self[p-1]}"') > -1:
                self.animation.add_edge(p, i)

    def _left(self, i, force=False) -> int | None:
        '''
        Gets the index of the left child of a heap element,
        handling the conversion betwee 0-indexing and 1-indexing

        :param int i: The would-be parent index of the return value
        :param bool force: Describes whether to only look in the heap (False),
            or whether to look in the entirety of self.ls
        :return: the index of i's left-child if it exists, or else, None
        :rtype: int | None
        '''

        idx = (i+1) * 2 - 1
        if force:
            return idx if idx < self.length else None
        else:
            return idx if idx < self.heapsize else None

    def _right(self, i, force=False) -> int | None:
        '''
        Gets the index of the right child of a heap element,
        handling the conversion betwee 0-indexing and 1-indexing.

        :param int i: The would-be parent index of the return value
        :param bool force: Describes whether to only look in the heap (False),
            or whether to look in the entirety of self.ls
        :return: the index of i's right-child if it exists, or else, None
        :rtype: int | None
        '''

        sibling = self._left(i, force=force)
        idx = sibling + 1 if sibling else None
        if not idx:
            return None
        if force:
            return idx if idx < self.length else None
        else:
            return idx if idx < self.heapsize else None

    def _parent(self, i, force=False) -> int | None:
        '''
        Gets the index of the parent of a heap element,
        handling the conversion betwee 0-indexing and 1-indexing.

        :param int i: The would-be left-or-right child index of the return
            value
        :param bool force: Describes whether to only look in the heap (False),
            or whether to look in the entirety of self.ls
        :return: the index of i's parent if it exists, or else, None
        :rtype: int | None
        '''

        idx = (i+1) // 2 - 1
        if force:
            return idx if idx < self.length and idx > -1 else None
        else:
            return idx if idx < self.heapsize and idx > -1 else None

    def _swap(self, i, j):
        self.retree_animation()
        self.animation.next_step()
        self.animation.highlight_edge(j,i)
        temp = self[i]
        self[i] = self[j]
        self[j] = temp
        self.retree_animation()
        self.animation.highlight_edge(j, i, color='green')

    def to_gif(self, size=500):
        digest = hashlib.sha256(str(self.ls).encode()).hexdigest()[:8]
        assets_dir = f'assets/gvanim/{self.classname()}/{digest}'
        basename = str.join('/', [assets_dir, 'frame'])
        gif_basename = f'{assets_dir}/{digest}'
        svg_basename = f'{assets_dir}/{digest}'
        files = None

        try:
            os.makedirs(assets_dir, exist_ok=False)
            files = render(self.animation.graphs(),
                           basename, fmt='png', size=size)
            render([self.animation.graphs()[-1]],
                   basename=svg_basename, fmt='svg', size=size)
            gif(files, gif_basename, size=size)
        except FileExistsError:
            pass

        last_state = f'{svg_basename}_000.svg'

        display_markdown('**Swapping process**', raw=True)
        display_markdown(f"![]({gif_basename}.gif)", raw=True)
        display_markdown('**Final result**', raw=True)
        display_markdown(f"![]({last_state})", raw=True)

    class SortDir(enum.Enum):
        ASC = 0,
        DSC = 1,


class MinHeap(Heap):
    def classname(self):
        return __class__.__name__

    def heap_prop(self, i, force=False) -> (bool, int | None):
        li = self._left(i, force=force)
        ri = self._right(i, force=force)
        idx = i
        if li and self[li] and self[li] < self[idx]:
            idx = li
        if ri and self[ri] and self[ri] < self[idx]:
            idx = ri
        # if not idx == i:
            # print(f'\t-> {
            #     self.classname()
            # } -> self[{idx}]: {self[idx]}, self[{i}]: {self[i]}')
        return (idx == i, idx)

    def heapsort(self, dir: Heap.SortDir | None = None):
        '''
        Sorts self.ls in descending order, and takes self.heapsize to 1

        :param Heap.SortDir|None: can be used to reverse the total order.
        '''
        super().heapsort()
        if dir == Heap.SortDir.ASC:
            list.reverse(self.ls)
            self.untree_animation()


class MaxHeap(Heap):
    def classname(self):
        return __class__.__name__

    def heap_prop(self, i, force=False) -> (bool, int | None):
        li = self._left(i, force=force)
        ri = self._right(i, force=force)
        idx = i
        if li and self[li] and self[li] > self[idx]:
            idx = li
        if ri and self[ri] and self[ri] > self[idx]:
            idx = ri
        # if not idx == i:
        #     print(f'\t-> {
        #         self.classname()
        #     } -> self[{idx}]: {self[idx]}, self[{i}]: {self[i]}')
        return (idx == i, idx)

    def heapsort(self, dir: Heap.SortDir | None = None):
        '''
        Sorts self.ls in ascending order, and takes self.heapsize to 1

        :param Heap.SortDir|None: can be used to reverse the total order.
        '''
        super().heapsort()
        if dir == Heap.SortDir.DSC:
            list.reverse(self.ls)
            self.untree_animation()


# Demonstration of properties
In the code/graphs below, you can see the properties of heaps come alive. If you uncomment the `interactive` statements, you can see how many steps it takes to each method called on the heap, which should be pretty accurate in terms of the time-complexity analysis presented in Chapter 6 of $CLRS$

In [2]:
from gvanim.jupyter import interactive

def maxheap(ls=[5,3,4,2,1,55,33,22,44,11]):
     return MaxHeap(ls)
    
def minheap(ls=[5,3,4,2,1,55,33,22,44,11]):
     return MinHeap(ls)

def demo(h:Heap, fx, *args, fmt='gif', i_size=800, **kwargs,):
    before = str(h)
    args = (h,) + args if args else h
    fx(args) if not kwargs else (fx, args, kwargs)
    after = str(h)
    h.retree_animation()
    print(f'before: {before}')
    print(f'after: {after}`')
    h.to_gif() if fmt == 'gif' else interactive(h.animation, i_size)

## MinHeap.build_heap() $\longrightarrow$ enforces the heap-property

$leftChild_{parent} \geq parent \land rightChild_{parent} \geq parent$

In [3]:
demo(minheap(), MinHeap.build_heap)
# demo(minheap(), MinHeap.build_heap, fmt='?')

Op: build_heap, MinHeap: [1, 2, 4, 5, 3, 55, 33, 22, 44, 11], heapsize: 10
before: MinHeap: [5, 3, 4, 2, 1, 55, 33, 22, 44, 11], heapsize: 0
after: MinHeap: [1, 2, 4, 5, 3, 55, 33, 22, 44, 11], heapsize: 10`


**Swapping process**

![](assets/gvanim/MinHeap/bfa3ba66/bfa3ba66.gif)

**Final result**

![](assets/gvanim/MinHeap/bfa3ba66/bfa3ba66_000.svg)

## MaxHeap.build_heap() $\longrightarrow$ enforces the heap-property
$leftChild_{parent} \leq parent \land rightChild_{parent} \leq parent$

In [4]:
demo(maxheap(), MaxHeap.build_heap)
# demo(maxheap(), MaxHeap.build_heap, fmt='?')

Op: build_heap, MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 10
before: MaxHeap: [5, 3, 4, 2, 1, 55, 33, 22, 44, 11], heapsize: 0
after: MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 10`


**Swapping process**

![](assets/gvanim/MaxHeap/90568b3e/90568b3e.gif)

**Final result**

![](assets/gvanim/MaxHeap/90568b3e/90568b3e_000.svg)

## MaxHeap.insert(8) $\longrightarrow$ insertion maintains heap-property

In [5]:
demo(maxheap(), Heap.insert, val=8)
# demo(maxheap(), Heap.insert, val=8, fmt='?')

before: MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 0
after: MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 0`


**Swapping process**

![](assets/gvanim/MaxHeap/90568b3e/90568b3e.gif)

**Final result**

![](assets/gvanim/MaxHeap/90568b3e/90568b3e_000.svg)

## MaxHeap.delete(indexOf(8)) $\longrightarrow$ deletion maintains heap-property

In [6]:
h=maxheap()
demo(h, MaxHeap.delete, i=h.ls.index(22))
# demo(h, MaxHeap.delete, i=h.ls.index(22), fmt='?')

before: MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 0
after: MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 0`


**Swapping process**

![](assets/gvanim/MaxHeap/90568b3e/90568b3e.gif)

**Final result**

![](assets/gvanim/MaxHeap/90568b3e/90568b3e_000.svg)

## MaxHeap.heapsort() $\longrightarrow$ total order (ascending)

In [7]:
demo(maxheap(), MaxHeap.heapsort)
# demo(maxheap(), MaxHeap.heapsort, fmt='?')

Op: build_heap, MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 10
Op: heapsort, MaxHeap: [1, 2, 3, 4, 5, 11, 22, 33, 44, 55], heapsize: 1
before: MaxHeap: [55, 44, 33, 22, 11, 4, 5, 3, 2, 1], heapsize: 0
after: MaxHeap: [1, 2, 3, 4, 5, 11, 22, 33, 44, 55], heapsize: 1`


**Swapping process**

![](assets/gvanim/MaxHeap/9b1a9e20/9b1a9e20.gif)

**Final result**

![](assets/gvanim/MaxHeap/9b1a9e20/9b1a9e20_000.svg)

## MinHeap.heapsort() $\longrightarrow$ total order (descending)

In [8]:
demo(minheap(), MinHeap.heapsort)
# demo(minheap(), MinHeap.heapsort, fmt='?')

Op: build_heap, MinHeap: [1, 2, 4, 5, 3, 55, 33, 22, 44, 11], heapsize: 10
Op: heapsort, MinHeap: [55, 44, 33, 22, 11, 5, 4, 3, 2, 1], heapsize: 1
before: MinHeap: [1, 2, 4, 5, 3, 55, 33, 22, 44, 11], heapsize: 0
after: MinHeap: [55, 44, 33, 22, 11, 5, 4, 3, 2, 1], heapsize: 1`


**Swapping process**

![](assets/gvanim/MinHeap/a3b7d4b2/a3b7d4b2.gif)

**Final result**

![](assets/gvanim/MinHeap/a3b7d4b2/a3b7d4b2_000.svg)